In [1]:
import numpy as np
import glob
import pandas as pd
import os

In [2]:
train_root_path='./car_license_dataset/train_images/training-set/'
test_root_path='./car_license_dataset/train_images/validation-set/'

dict_number_value=list('ABCDEFGHJKLMNPQRSTUVWXYZ')
dict_number_key=list(range(10,34))
dict_number=dict(zip(dict_number_key,dict_number_value))
dict_number

{10: 'A',
 11: 'B',
 12: 'C',
 13: 'D',
 14: 'E',
 15: 'F',
 16: 'G',
 17: 'H',
 18: 'J',
 19: 'K',
 20: 'L',
 21: 'M',
 22: 'N',
 23: 'P',
 24: 'Q',
 25: 'R',
 26: 'S',
 27: 'T',
 28: 'U',
 29: 'V',
 30: 'W',
 31: 'X',
 32: 'Y',
 33: 'Z'}

In [3]:
train_paths=glob.glob(train_root_path+'*/*')
test_paths=glob.glob(test_root_path+'*/*')

In [4]:
number_paths=[]
labels=[]
for path in train_paths:
    idx=path.split('/')[-2]
    if str.isdigit(idx):
        if 9<int(idx)<34:
            label=dict_number[int(idx)]
            number_paths.append(path)
            labels.append(label)

In [5]:
train_number_data=np.c_[np.array(number_paths),np.array(labels)]
np.random.shuffle(train_number_data)
number_train_data=pd.DataFrame(train_number_data,columns=['paths','labels'])
number_train_data

,paths,labels
0,/root/tensorflow_practice/carplate_recognition...,A
1,/root/tensorflow_practice/carplate_recognition...,C
2,/root/tensorflow_practice/carplate_recognition...,N
3,/root/tensorflow_practice/carplate_recognition...,B
4,/root/tensorflow_practice/carplate_recognition...,M
...,...,...
3193,/root/tensorflow_practice/carplate_recognition...,U
3194,/root/tensorflow_practice/carplate_recognition...,C
3195,/root/tensorflow_practice/carplate_recognition...,C
3196,/root/tensorflow_practice/carplate_recognition...,N


In [6]:
number_paths=[]
labels=[]
for path in test_paths:
    idx=path.split('/')[-2]
    if str.isdigit(idx):
        if 9<int(idx)<34:
            label=dict_number[int(idx)]
            number_paths.append(path)
            labels.append(label)

In [7]:
test_number_data=np.c_[np.array(number_paths),np.array(labels)]
np.random.shuffle(test_number_data)
number_test_data=pd.DataFrame(test_number_data,columns=['paths','labels'])
number_test_data

,paths,labels
0,/root/tensorflow_practice/carplate_recognition...,A
1,/root/tensorflow_practice/carplate_recognition...,A
2,/root/tensorflow_practice/carplate_recognition...,M
3,/root/tensorflow_practice/carplate_recognition...,Q
4,/root/tensorflow_practice/carplate_recognition...,B
...,...,...
76,/root/tensorflow_practice/carplate_recognition...,K
77,/root/tensorflow_practice/carplate_recognition...,G
78,/root/tensorflow_practice/carplate_recognition...,B
79,/root/tensorflow_practice/carplate_recognition...,G


In [8]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [9]:
height = 32
width = 32
channels = 1
batch_size = 32
num_classes = 24

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_dataframe(
    number_train_data,
    directory = '/',
    x_col = 'paths',
    y_col = 'labels',
    classes = dict_number_value,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = True,
    class_mode = 'sparse',
    color_mode='grayscale'
)

Found 3198 validated image filenames belonging to 24 classes.


In [10]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255
)

valid_generator = valid_datagen.flow_from_dataframe(
    number_test_data,
    directory = '/',
    x_col = 'paths',
    y_col = 'labels',
    classes = dict_number_value,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = "sparse",
color_mode='grayscale')

Found 81 validated image filenames belonging to 24 classes.


In [11]:
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(32, 32, 32, 1) (32,)
[ 2. 17. 17.  5. 22.  3.  0.  9. 19. 23. 15. 19. 23.  2. 20.  1.  2. 19.
  1.  2. 20.  4.  0.  0. 22. 17.  1.  1.  7.  9.  0. 13.]
(32, 32, 32, 1) (32,)
[ 4. 16. 16.  9. 11.  2.  1. 19.  4. 14. 14. 16. 21. 22.  0. 18.  4.  2.
 15.  0.  0. 15.  0. 17. 12.  0. 14.  0. 15. 12. 15.  6.]


In [14]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same',
                        activation='relu', 
                        input_shape=[width, height, channels]),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(num_classes, activation='softmax'),
])

initial_lr=0.01
epochs = 20
decay_rate=initial_lr/epochs
sgd=keras.optimizers.SGD(lr=initial_lr,decay=decay_rate)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=sgd, metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 128)        

In [15]:
train_num = train_generator.samples
valid_num = valid_generator.samples

history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num // batch_size)

Epoch 1/20
99/99 [==============================] - 21s 216ms/step - loss: 0.4242 - accuracy: 0.9106 - val_loss: 2.9546 - val_accuracy: 0.2031
Epoch 2/20
99/99 [==============================] - 21s 212ms/step - loss: 0.0208 - accuracy: 0.9994 - val_loss: 2.6302 - val_accuracy: 0.3281
Epoch 3/20
99/99 [==============================] - 20s 204ms/step - loss: 0.0123 - accuracy: 0.9994 - val_loss: 1.7061 - val_accuracy: 0.6875
Epoch 4/20
99/99 [==============================] - 20s 207ms/step - loss: 0.0086 - accuracy: 0.9997 - val_loss: 0.5407 - val_accuracy: 1.0000
Epoch 5/20
99/99 [==============================] - 20s 206ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0893 - val_accuracy: 1.0000
Epoch 6/20
99/99 [==============================] - 21s 207ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 1.0000
Epoch 7/20
99/99 [==============================] - 21s 212ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000